In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb
from checklist.pred_wrapper import PredictorWrapper
import numpy as np
import random
import torch
from transformers import pipeline
import json
import requests
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
from scipy import spatial
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from collections import Counter

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [9]:
response = requests.get("https://square.ukp-lab.de/api/skill-manager/skill")
skills = response.json()
selected_skills = []
for skill in skills:
    if "open" in skill['name'].lower() and "bio" not in skill['name'].lower():
    #if  "bio" in skill['name'].lower():
        selected_skills.append(skill)
selected_skills = [selected_skills[0], selected_skills[2]]
length_of_skills = len(selected_skills)
print(length_of_skills)
print(selected_skills)

2
[{'id': '62c1ae1f536b1bb18ff91ced', 'name': 'OpenSQuAD-DPR', 'url': 'http://open-extractive-qa', 'skill_type': 'span-extraction', 'skill_settings': {'requires_context': False, 'requires_multiple_choices': 0}, 'user_id': 'ukp', 'created_at': '2022-07-05T07:56:01.070904', 'skill_input_examples': [{'query': '', 'context': '', 'answers': None}, {'query': '', 'context': '', 'answers': None}, {'query': '', 'context': '', 'answers': None}], 'description': 'An open-domain QA skill retrieving from Wikipedia (DPR) with bert-base-uncased as base reader model and the bert-base-uncased-pf-squad_v2 adapter.', 'default_skill_args': {'base_model': 'bert-base-uncased', 'adapter': 'AdapterHub/bert-base-uncased-pf-squad_v2', 'datastore': 'nq', 'index': 'dpr'}, 'published': True, 'client_id': 'ukp-OpenSQuAD-DPR', 'client_secret': None}, {'id': '62c2a1f93f28686f3c9d9363', 'name': 'OpenSQuAD-BM25', 'url': 'http://open-extractive-qa', 'skill_type': 'span-extraction', 'skill_settings': {'requires_context': 

In [10]:
data_file = "nqopen_selected.txt"
json_dicts = []
json_file_names = []
for i in range(length_of_skills):
    json_dicts.append(dict())
    json_dicts[i]["tests"] = []
    file_name = selected_skills[i]['id'] +'.json'
    json_file_names.append(file_name)

In [11]:
capabilities = {
    'Vocabulary': 'This capability tests whether a model has necessary vocabulary and whether it has the ability to handle the importance of different words.',
    'Taxonomy': 'This capability tests whether a model has the ability to handle synonyms, antonyms, object size, shape, profession etc.',
    'Robustness': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
    'NER': 'NER (Named Entity Recognition) Tests model’s ability to appropriately understand named entities.',
    'Negation': "This capability test's a model's ability to handle negation",
    'Temporal': "This capability test's a model's ability to understand order of events.",
    'Fairness': "This capability test's the model's gender(M/F) or racial bias",
    'Coref':"Coref (Coreference) Test's model's ability to appropriately understand he / she.",
    'SRL': "SRL (Semantic Role Labeling) Test's model's capability to understand different roles such as agents and objects."
    
}
test_types = {
    'MFT': 'MFT stands for Minimum Functionality Test. This testing type is inspired from unit testing of software engineering. For this type of testing precise and small testing datasets are created and the models are tested on that particular test set. MFTs are useful particularly for detecting when models use alternative approaches to handle complicated inputs without actually knowing the inside out of the capability. For MFT test cases, labeled test set is required.',
    'INV': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.'
}

In [12]:
def pred_answer(question, selected_skill_index):
    try:
        skill = selected_skills[selected_skill_index]
        skill_id = skill['id']
        query = {
            "query": question,
            "skill_args": {
                "base_model": skill['default_skill_args']['base_model'],
                "adapter": skill['default_skill_args']['adapter']
            },
        }
        headers = {'Content-type': 'application/json'}
        json_object = json.dumps(query)
        response = requests.post("https://square.ukp-lab.de/api/skill-manager/skill/"+str(skill_id)+"/query", data = json_object, headers = headers)
        predictions = response.json()
        return predictions['predictions'][0]['prediction_output']['output']
    except Exception as ex:
        return "failed"

In [14]:
pred_answer("where is bangladesh", 1)

'by the armed forces of the commonwealth nations'

In [15]:
nq_open = open(data_file, 'r')
questions = []
answers_nq = []
for line in nq_open:
    line = line.strip()
    elements = line.split("|||||")
    questions.append(elements[0])
    answers_nq.append(elements[1:])

In [16]:
questions

['when was the last time anyone was on the moon ',
 "who wrote he ain't heavy he's my brother lyrics ",
 'who does the voice of nala in the lion king ',
 'what age do you need to be to buy a bb gun ',
 'who plays auggie in the movie the wonder ',
 'when was the first australian prime minister elected ',
 'who plays joker in batman the dark knight ',
 'when was the south asian association for regional co-operation (saarc) formed ',
 'when was the first hunger games book published ',
 'who did bette midler portray in the rose ',
 'who wrote cant get you out of my head lyrics ',
 'what are the three fifty shades of grey books ',
 'who is the coach for the ottawa senators ',
 'what is the name of the main artery which takes blood from the heart to the body ',
 'who does eric end up with in that 70s show ',
 'who wrote papa got a brand new bag ',
 'who are nominated for president of india 2017 ',
 'where does the path train stop in newark ',
 'where does route 66 start on the west coast ',


In [17]:
def get_answer(question):
    for i in range(len(questions)):
        if questions[i].strip() == question.strip():
            return answers_nq[i]

In [18]:
contraction_map = {
            "isn't": "is not", "ain't": "is not", "aren't": "are not", "can't": "cannot",
            "can't've": "cannot have", "could've": "could have", "couldn't":
            "could not", "didn't": "did not", "doesn't": "does not", "don't":
            "do not", "hadn't": "had not", "hasn't": "has not", "haven't":
            "have not", "he'd": "he would", "he'd've": "he would have",
            "he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y":
            "how do you", "how'll": "how will", "how's": "how is",
            "I'd": "I would", "I'll": "I will", "I'm": "I am",
            "I've": "I have", "i'd": "i would", "i'll": "i will",
            "i'm": "i am", "i've": "i have",
            "it'd": "it would", "it'll": "it will", "it's": "it is", "ma'am":
            "madam", "might've": "might have", "mightn't": "might not",
            "must've": "must have", "mustn't": "must not", "needn't":
            "need not", "oughtn't": "ought not", "shan't": "shall not",
            "she'd": "she would", "she'll": "she will", "she's": "she is",
            "should've": "should have", "shouldn't": "should not", "that'd":
            "that would", "that's": "that is", "there'd": "there would",
            "there's": "there is", "they'd": "they would",
            "they'll": "they will", "they're": "they are",
            "they've": "they have", "wasn't": "was not", "we'd": "we would",
            "we'll": "we will", "we're": "we are", "we've": "we have",
            "weren't": "were not", "what're": "what are", "what's": "what is",
            "when's": "when is", "where'd": "where did", "where's": "where is",
            "where've": "where have", "who'll": "who will", "who's": "who is",
            "who've": "who have", "why's": "why is", "won't": "will not",
            "would've": "would have", "wouldn't": "would not",
            "you'd": "you would", "you'd've": "you would have",
            "you'll": "you will", "you're": "you are", "you've": "you have"
}



In [26]:
def contractions(x):
    conts = Perturb.contractions(x)
    return conts
t = Perturb.perturb(questions, contractions, nsamples=5)

In [27]:
t.data

[['who is the first president to be impeached ',
  "who's the first president to be impeached "],
 ['what is the name of the hyena in lion king ',
  "what's the name of the hyena in lion king "],
 ['what is the oldest street in the philippines ',
  "what's the oldest street in the philippines "],
 ['who is the king and queen of the netherlands ',
  "who's the king and queen of the netherlands "],
 ['what is the meaning of cc and bcc ', "what's the meaning of cc and bcc "]]

In [28]:
import re
keys = list(contraction_map.keys())
values = list(contraction_map.values())
changes = []
for i in range(len(t.data)):
    actual_sentence =  t.data[i][0]
    changed_sentence = t.data[i][1]
    #print(actual_sentence)
    #print(changed_sentence)
    key_words = [] 
    for k in keys:
        r = re.search(k,changed_sentence, re.IGNORECASE)
        if bool(r) == True and r.group() not in key_words:
            if changed_sentence[r.span(0)[1]] == ' ':
                key_words.append(r.group())

    value_words = []
    for k in values:
        r = re.search(k, changed_sentence, re.IGNORECASE)
        if bool(r) == True and r.group() not in value_words:
            if changed_sentence[r.span(0)[1]] == ' ':
                value_words.append(r.group())
    all_words = key_words + value_words
    #print(all_words)
    all_values = []
    all_values_ = []
    for w in all_words:
        if w in keys:
            all_values.append(contraction_map[w])
        elif w.capitalize() in keys:
            all_values.append(contraction_map[w.capitalize()])
        elif w.lower() in keys:
            all_values.append(contraction_map[w.lower()])
        elif w in values:
            for j in range(len(values)):
                if w.strip() == values[j] and keys[j] not in all_values:
                    all_values.append(keys[j])
        elif w.capitalize() in values:
            for j in range(len(values)):
                if w.capitalize() == values[j] and keys[j] not in all_values:
                    all_values.append(keys[j])
        elif w.lower() in values:
            for j in range(len(values)):
                if w.lower() == values[j] and keys[j] not in all_values:
                    all_values.append(keys[j])
    actual = []
    change = []
    for i in range(len(all_words)):
        try:
            r = re.search(all_values[i],actual_sentence, re.IGNORECASE)
            if r.group() not in all_values_:
                actual.append(r.group())
            change.append(all_words[i])
        except:
            continue
    changed_pairs = []
    for m in range(len(actual)):
        o, c = actual[m], change[m]
        changed_pairs.append((o, c))
    changes.append(changed_pairs)

In [29]:
model = SentenceTransformer('all-MiniLM-L6-v2')
thresholds = [0.5, 0.5, 0.7,0.5]

def jaccard_similarity(x, y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def tfidf_similarity(corpus):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim[0][1]

def sbert_similarity(sentence, changed_sentence):
    sentences = [sentence, changed_sentence]
    embeddings = model.encode(sentences)
    result = 1 - spatial.distance.cosine(embeddings[0], embeddings[1])
    return result

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    #def remove_articles(text):
    #    return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(ground_truth, prediction):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def answer_or_not(sentence1, sentence2, thresholds):
    sentence1 = normalize_answer(sentence1)
    sentence2 = normalize_answer(sentence2)
    corpus = [sentence1, sentence2]
    s1_splitted = sentence1.split(" ")
    s2_splitted = sentence2.split(" ")
    scores = []
    scores.append(jaccard_similarity(s1_splitted, s2_splitted))
    scores.append(tfidf_similarity(corpus))
    scores.append(sbert_similarity(sentence1, sentence2))
    scores.append(f1_score(sentence1, sentence2))
    got_it = []
    for i in range(len(scores)):
        if scores[i]>thresholds[i]:
            got_it.append(1)
        else:
            got_it.append(0)
    if sum(got_it) >= 3:
        return 1
    else:
        return 0

In [30]:
def add_test(data, changes, test_name, capability, test_type, name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q = data[i][0]
            changed_q = data[i][1]
            change = changes[i]
            a = pred_answer(original_q, skill_index)
            a = normalize_answer(a.strip().lower())
            p = pred_answer(changed_q, skill_index)
            p = normalize_answer(p.strip().lower())
            test_case_dict = dict()
            total_cases = total_cases + 1 
            test_case_dict["original_question"] = original_q
            test_case_dict["question"] = changed_q
            answers = get_answer(original_q)
            for n in range(len(answers)):
                answers[n] = answers[n].strip().lower()
            test_case_dict["gold_answer"] = answers
            test_case_dict["prediction_before_change"] = a
            test_case_dict["prediction"] = p
            changed_dict = dict()
            changed_dict["from"] = []
            changed_dict["to"] = []
            for n in range(len(change)):
                source_target_pairs = change[n]
                s, t = source_target_pairs
                changed_dict["from"].append(s)
                changed_dict["to"].append(t)
            test_case_dict["changed"] = changed_dict
            feedback = answer_or_not(p, a, thresholds)
            if feedback == 0:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                test_case_dict["success_failed"] = "success"
            '''
            if a != p:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                if a == "Failed" and p == "Failed":
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
            '''
            test_case_dict["where"] = "question"
            test_cases.append(test_case_dict)
            
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)

    return 0

In [31]:
name='Question contractions'
test_type = "INV"
capability='Robustness'
thresholds = [0.25, 0.25, 0.5, 0.25]
n_description = "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(t.data, changes, name, capability, test_type, n_description, t_description, c_description)

0

In [32]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'under article 1 what is the minimum age required to serve in the house of representatives ',
    'question': "under article 1 what's the minimum 

In [17]:
def addtypos(x):
    conts = Perturb.add_typos(x)
    return conts
t = Perturb.perturb(questions, addtypos, nsamples=10)

In [18]:
t.data

[['why does kerosene oil rise up in the wick of lantern ',
  'why does kerosene oil rsie up in the wick of lantern '],
 ['who captained the first european ship to sail around the tip of africa ',
  'who captained the first european ship to sail around the ti pof africa '],
 ['when did they stop cigarette advertising on television ',
  'whne did they stop cigarette advertising on television '],
 ['when did the rational dress society begin to work ',
  'when did the rational dress societ ybegin to work '],
 ['when was the first hunger games book published ',
  'when was the first hunger games boo kpublished '],
 ['when do you celebrate birthday if born on feb 29 ',
  'when do you celebrate birthdayi f born on feb 29 '],
 ['who is the first president to be impeached ',
  'whoi s the first president to be impeached '],
 ['what is the name of the hyena in lion king ',
  'what is the name o fthe hyena in lion king '],
 ['what is the oldest street in the philippines ',
  'what is the oldest s

In [19]:
changed_pairs = []
for i in range(len(t.data)):
   
    actual_sentence = t.data[i][0]
    changed_sentence = t.data[i][1]
    actual_list = actual_sentence.split()
    changed_list = changed_sentence.split()
    a_list = []
    for j in range(len(changed_list)):
        a = actual_list[j]
        c = changed_list[j]
        if a != c:
            pair = (a, c)
            a_list.append(pair)
    changed_pairs.append(a_list)

In [20]:
name='Question typo'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle questions typos (whether changing the spelling of words in the questions changes the model's output)"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(t.data, changed_pairs, name, capability, test_type, n_description, t_description, c_description)

0

In [21]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'under article 1 what is the minimum age required to serve in the house of representatives ',
    'question': "under article 1 what's the minimum 

In [22]:
nlp = spacy.load('en_core_web_sm')

In [23]:
from datasets import load_dataset
dataset = load_dataset('squad_v2', split='validation')
new_questions = [d['question'] for d in dataset]
contexts = [d['context'] for d in dataset]
answers = []
i = 0
pairs = []
for single_data in dataset:
    try:
        c = single_data["context"]
        q = single_data["question"]
        a = single_data["answers"]["text"]
        if bool(a[0].strip()) == True:
            pairs.append((c, q))
            answers.append(a)
    except:
        i = i+1
        #answers.append(["Not a Answerable Question"])
print(i)

Reusing dataset squad_v2 (/home/saadi/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


5945


In [24]:
from tqdm import tqdm
contexts = []
new_questions = []
for p in pairs:
    c, q = p
    contexts.append(c)
    new_questions.append(q)
qa = list(zip(new_questions, contexts))

all_qa = set()
all_qa.update(set(new_questions))
all_qa.update(set(contexts))

processed_qa = list(tqdm(nlp.pipe(all_qa, batch_size=64), total=len(all_qa)))
spacy_map = {q: processed_q for (q, processed_q) in zip(all_qa, processed_qa)}
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]

100%|██████████████████████████████████████| 7130/7130 [00:18<00:00, 376.16it/s]


In [25]:
random_sentences = set()
for x, _ in processed_pairs:
    for y in x.sents:
        random_sentences.add(y.text)
random_sentences = list(random_sentences)

In [26]:
def add_random_sentence(x, **kwargs):
    random_s = np.random.choice(random_sentences)
    while random_s in x[0]:
        random_s = np.random.choice(random_sentences)
    random_s = random_s.strip('.') + '. '
    meta = ['add to end: %s' % random_s, 'add to beg: %s' % random_s]
    return [x +  " " + random_s, random_s + " " + x], meta

In [27]:
t = Perturb.perturb(questions , add_random_sentence, nsamples=2, meta=True)

In [28]:
def add_random_sentence_test(data, changes, test_name, capability, test_type, 
                             name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q = data[i][0]
            temp_data = data[i][1:]
            temp_changes = changes[i][1:]
            for j in range(len(temp_data)):
                changed_q = temp_data[j]
                random_sentence = temp_changes[j] 
                a = pred_answer(original_q, skill_index)
                a = normalize_answer(a.strip().lower())
                p = pred_answer(changed_q, skill_index)
                p = normalize_answer(p.strip().lower())
                test_case_dict = dict()
                test_case_dict["original_question"] = original_q
                test_case_dict["question"] = changed_q
                test_case_dict["prediction_before_change"] = a
                answers = get_answer(original_q)
                for n in range(len(answers)):
                    answers[n] = answers[n].strip().lower()
                test_case_dict["gold_answer"] = answers
                test_case_dict["prediction"] = p
                temp = changed_q.replace(original_q, '')
                start = changed_q.find(temp)
                end = start + len(temp)
                span = [start, end]
                test_case_dict["span"] = span
                feedback = answer_or_not(p, a, thresholds)
                if feedback == 0:
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
                total_cases = total_cases + 1        
                test_case_dict["where"] = "question"
                test_cases.append(test_case_dict)
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)
    
    return 0

In [29]:
name = 'Add random sentence to context'
capability='Robustness'
test_type="INV"
n_description = "Test's model's ability to handle addition of random sentences in the model's input context (whether addition of a random sentence at the beginning and end of the input context changes model's output)."
c_description = capabilities[capability]
t_description = test_types[test_type]
add_random_sentence_test(t.data, t.meta, name, capability, test_type, n_description, t_description, c_description)

0

In [30]:
json_dicts[0]

{'tests': [{'test_type': 'INV',
   'capability': 'Robustness',
   'test_name': 'Question contractions',
   'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
   'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
   'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
   'test_cases': [{'original_question': 'under article 1 what is the minimum age required to serve in the house of representatives ',
     'question': "under article 1 wh

In [31]:
new_questions = questions.copy()
for i in range(len(new_questions)):
    new_questions[i] = new_questions[i].upper()
fake_context = [" "]*len(new_questions)

In [32]:
fake_pairs = []
for fc, q in zip(fake_context, new_questions):
    fake_pairs.append((fc, q))

In [33]:
qa = list(zip(new_questions, fake_context))

all_qa = set()
all_qa.update(set(new_questions))
all_qa.update(set(fake_context))

In [34]:
processed_qa = list(tqdm(nlp.pipe(all_qa, batch_size=64), total=len(all_qa)))
spacy_map = {q: processed_q for (q, processed_q) in zip(all_qa, processed_qa)}
processed_pairs = [spacy_map[x[1]] for x in fake_pairs]

100%|██████████████████████████████████████████| 65/65 [00:00<00:00, 910.45it/s]


In [35]:
import re
def change_thing(change_fn):
    def change_both(q, **kwargs):
        #context, question = cq
        a = change_fn(q, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            #new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append(c)
        return ret, meta
    return change_both
            

In [36]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_names), nsamples=2, meta=True)

In [37]:
t.data

[['WHO DOES ERIC END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Jeremiah END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Austin END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Jordan END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Jordan END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Derek END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Adam END UP WITH IN THAT 70S SHOW ',
  'WHO DOES John END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Samuel END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Michael END UP WITH IN THAT 70S SHOW ',
  'WHO DOES Jayden END UP WITH IN THAT 70S SHOW '],
 ['WHO WAS ORIGINALLY CAST TO PLAY INDIANA JONES ',
  'WHO WAS ORIGINALLY CAST TO PLAY Jennifer Sullivan ',
  'WHO WAS ORIGINALLY CAST TO PLAY Jessica Hill ',
  'WHO WAS ORIGINALLY CAST TO PLAY Ashley Green ',
  'WHO WAS ORIGINALLY CAST TO PLAY Sarah Turner ',
  'WHO WAS ORIGINALLY CAST TO PLAY Emily Lewis ',
  'WHO WAS ORIGINALLY CAST TO PLAY Amanda Gomez ',
  'WHO WAS ORIGINALLY CAST TO PLAY Elizabeth Scott ',
  'WHO WAS ORIGINALLY CAST

In [38]:
def add_ner(data, changes, test_name, capability, test_type, 
            name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q = data[i][0]
            temp_data = data[i][1:]
            temp_changes = changes[i][1:]
            original_q = original_q.lower()
            a = pred_answer(original_q, skill_index)
            a = normalize_answer(a.strip().lower())
            for j in range(len(temp_data)):
                changed_q = temp_data[j]
                changed_q = changed_q.lower()
                s, t = temp_changes[j]
                p = pred_answer(changed_q, skill_index)
                p = normalize_answer(p.strip().lower())
                test_case_dict = dict()
                total_cases = total_cases + 1  
                test_case_dict["original_question"] = original_q
                test_case_dict["question"] = changed_q
                test_case_dict["prediction_before_change"] = a
                answers = get_answer(original_q.lower())
                for n in range(len(answers)):
                    answers[n] = answers[n].strip().lower()
                test_case_dict["gold_answer"] = answers
                test_case_dict["prediction"] = p
                changed_dict = dict()
                changed_dict["from"] = s.lower()
                changed_dict["to"] = t.lower()
                test_case_dict["changed"] = changed_dict
                feedback = answer_or_not(p, a, thresholds)
                if feedback == 0:
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
                test_case_dict["where"] = "question"
                test_cases.append(test_case_dict)
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)
    return 0

In [39]:
t.meta

[[None,
  ('ERIC', 'Jeremiah'),
  ('ERIC', 'Austin'),
  ('ERIC', 'Jordan'),
  ('ERIC', 'Jordan'),
  ('ERIC', 'Derek'),
  ('ERIC', 'Adam'),
  ('ERIC', 'John'),
  ('ERIC', 'Samuel'),
  ('ERIC', 'Michael'),
  ('ERIC', 'Jayden')],
 [None,
  ('INDIANA JONES', 'Jennifer Sullivan'),
  ('INDIANA JONES', 'Jessica Hill'),
  ('INDIANA JONES', 'Ashley Green'),
  ('INDIANA JONES', 'Sarah Turner'),
  ('INDIANA JONES', 'Emily Lewis'),
  ('INDIANA JONES', 'Amanda Gomez'),
  ('INDIANA JONES', 'Elizabeth Scott'),
  ('INDIANA JONES', 'Melissa Young'),
  ('INDIANA JONES', 'Stephanie Hall'),
  ('INDIANA JONES', 'Nicole Wright')]]

In [40]:
name = 'Change name everywhere'
capability='NER'
test_type="INV"
n_description = "Test's model's capability to handle changing a name in the input context and also in the question (if it contains the name)(whether a name change in the context and the questions changes the model output)."
c_description = capabilities[capability]
t_description = test_types[test_type]
add_ner(t.data, t.meta, name, capability, test_type, n_description, t_description, c_description)

0

In [41]:
json_dicts[0]

{'tests': [{'test_type': 'INV',
   'capability': 'Robustness',
   'test_name': 'Question contractions',
   'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
   'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
   'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
   'test_cases': [{'original_question': 'under article 1 what is the minimum age required to serve in the house of representatives ',
     'question': "under article 1 wh

In [42]:
new_questions = questions.copy()
for i in range(len(new_questions)):
    new_questions[i] = new_questions[i].title()
fake_context = [" "]*len(new_questions)

In [43]:
fake_pairs = []
for fc, q in zip(fake_context, new_questions):
    fake_pairs.append((fc, q))

In [44]:
qa = list(zip(new_questions, fake_context))

all_qa = set()
all_qa.update(set(new_questions))
all_qa.update(set(fake_context))

In [45]:
processed_qa = list(tqdm(nlp.pipe(all_qa, batch_size=64), total=len(all_qa)))
spacy_map = {q: processed_q for (q, processed_q) in zip(all_qa, processed_qa)}
processed_pairs = [spacy_map[x[1]] for x in fake_pairs]

100%|██████████████████████████████████████████| 65/65 [00:00<00:00, 779.26it/s]


In [46]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_location), nsamples=2, meta=True)

In [47]:
t.meta

[[None,
  ('India', 'Angola'),
  ('India', 'Vietnam'),
  ('India', 'South Africa'),
  ('India', 'Ukraine'),
  ('India', 'United Kingdom'),
  ('India', 'Thailand'),
  ('India', 'Pakistan'),
  ('India', 'South Africa'),
  ('India', 'Afghanistan'),
  ('India', 'France')],
 [None,
  ('Philippines', 'Bangladesh'),
  ('Philippines', 'Russian Federation'),
  ('Philippines', 'Egypt'),
  ('Philippines', 'Brazil'),
  ('Philippines', 'China'),
  ('Philippines', 'Russian Federation'),
  ('Philippines', 'Germany'),
  ('Philippines', 'Germany'),
  ('Philippines', 'Ethiopia'),
  ('Philippines', 'Pakistan')]]

In [48]:
name = 'Change location everywhere'
capability='NER'
test_type="INV"
n_description = "Test's model's capability to handle changing a location in the input context and also in the question (if it contains the name)(whether a location change in the context and the question changes the model output)."
c_description = capabilities[capability]
t_description = test_types[test_type]
add_ner(t.data, t.meta, name, capability, test_type, n_description, t_description, c_description)

KeyboardInterrupt: 

In [ ]:
json_dicts[0]

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
paraphraser = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
paraphraser = paraphraser.to(device)
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

In [ ]:
i = 0
paraphrased_questions = []
for q in questions:
    batch = tokenizer(q, return_tensors='pt')
    generated_ids = paraphraser.generate(batch['input_ids'])
    generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_sentence = generated_sentence.replace('?', '').lower()
    if q.strip() == generated_sentence:
        continue
    result = sbert_similarity(q, generated_sentence)
    if result >= 0.89:
        paraphrased_questions.append([q, generated_sentence])
        i = i + 1
    else:
        continue
    if i == 10:
        break

In [ ]:
def add_paraphrased_test(data, test_name, capability, test_type, name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q, changed_q= data[i][0], data[i][1]
            a = pred_answer(original_q, skill_index)
            a = normalize_answer(a)
            p = pred_answer(changed_q, skill_index)
            p = p.strip().lower()
            test_case_dict = dict()
            total_cases = total_cases + 1 
            test_case_dict["original_question"] = original_q
            test_case_dict["question"] = changed_q
            answers = get_answer(original_q)
            for n in range(len(answers)):
                answers[n] = answers[n].strip().lower()
            test_case_dict["gold_answer"] = answers
            test_case_dict["prediction_before_change"] = a
            test_case_dict["prediction"] = p
            feedback = answer_or_not(p, a, thresholds)
            if feedback == 0:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                test_case_dict["success_failed"] = "success"
            '''
            if a != p:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                if a == "Failed" and p == "Failed":
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
            '''
            test_case_dict["where"] = "question"
            test_cases.append(test_case_dict)
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)
    return 0

In [ ]:
name = 'Paraphrase Question'
capability='Robustness'
test_type="INV"
n_description = "Test's model's capability to handle paraphrased question"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_paraphrased_test(paraphrased_questions, name, capability, test_type, n_description, t_description, c_description)

In [ ]:
json_dicts[0]["tests"]

In [ ]:
def shuffle_question(question):
    random.seed(0)
    question = question.replace('?','')
    question = question.replace('.','')
    splitted_question = question.split()
    word_range = range(0,len(splitted_question))
    selected_indexes = random.sample(word_range, 2)
    word = splitted_question[selected_indexes[0]]
    word = wordword = splitted_question[selected_indexes[0]]
    splitted_question[selected_indexes[0]] = splitted_question[selected_indexes[1]]
    splitted_question[selected_indexes[1]] = word
    return ' '.join(splitted_question)

In [ ]:
nsamples = 10
i = 0
changes = []
shuffuled_questions = []
j = 0
for i in range(len(questions)):
    q = questions[i]
    c_q = shuffle_question(q)
    result = sbert_similarity(q, c_q)
    print(q)
    print(c_q)
    c_in_q = []
    q_split = q.split()
    c_split = c_q.split()
    for n in range(len(q.split())):
        r = q_split[n]
        c = c_split[n]
        if r != c:
            c_in_q.append((r, c))
    if result >= 0.92:
        shuffuled_questions.append([q, c_q])
        changes.append(c_in_q)
        j = j + 1
    if j == nsamples:
        break

In [ ]:
changes

In [ ]:
name='Perturb some words'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle unorderd questions "
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(shuffuled_questions, changes, name, capability, test_type, n_description, t_description, c_description)

In [ ]:
def process_questions(ques):
    changes = []
    ans = []
    questions_ = []
    for i in range(len(ques)):
        q = ques[i]
        elements = q.split("|||||")
        splited = elements[0].split()
        oq = []
        cq = []
        change = []
        for j in range(len(splited)):
            if "/" not in splited[j]:
                oq.append(splited[j])
                cq.append(splited[j])
            else:
                temp = splited[j].split("/")
                oq.append(temp[0])
                cq.append(temp[1])
                change.append((temp[0], temp[1]))
        original = ' '.join([str(w) for w in oq])
        changed = ' '.join([str(w) for w in cq])
        questions_.append([original, changed])
        changes.append(change)
        ans.append(elements[1:])
    return questions_, changes, ans

In [ ]:
#synonym questions nq
synonyms = ["who was the famous/renowned scientist that ran/directed the research lab moseley went to in manchester |||||sir ernest rutherford",
"when was the first hunger games book published/released |||||september 14 2008|||||2008",
"when was the minimum wage/salary established/setlled in the united states |||||1938|||||1933|||||1912",
"when did the rational dress society begin/start to work |||||1881",
"what is the oldest/earliest street/road in the philippines |||||cebu city|||||colon street",
"who was originally/actualy cast to play/act indiana jones |||||tom selleck",
"who made/created the song falling in love with you |||||hugo peretti|||||george david weiss|||||elvis presley|||||luigi creatore",
"which country has the most coastline/shoreline in the world |||||canada",
"who captained/commanded the first european ship to sail around the tip of africa |||||bartolomeu dias",
"who is currently/now serving as president of the senate |||||orrin hatch|||||patrick leahy"]

final_questions, final_changes, ans = process_questions(synonyms)

In [ ]:
json_dicts[0]["tests"]

In [ ]:
name='Replace some words with synonyms'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle synonyms in questions"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(final_questions, final_changes, name, capability, test_type, n_description, t_description, c_description)

In [ ]:
#antonym questions nq
antonyms = ["who plays joker in batman the dark/bright knight |||||ledger",
"where does route 66 start/end on the west/east coast |||||in santa monica",
"who did bette midler portray/misportray in the rose |||||mary rose foster",
"who is the first/last president to be impeached |||||andrew johnson|||||johnson",
"which country has the most/least coastline in the world |||||canada",
"when was the first/last australian prime minister elected |||||sir edmund barton|||||1901",
"what is the oldest/newest street in the philippines |||||cebu city|||||colon street"]
final_questions, final_changes, ans = process_questions(antonyms)

In [ ]:
name='Replace some words with antonym in the question'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle antonyms in questions"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(final_questions, final_changes, name, capability, test_type, n_description, t_description, c_description)

In [ ]:
json_dicts[0]["tests"]

when was the last time anyone was on the moon |||||14 december 1972 utc|||||december 1972
who wrote he ain't heavy he's my brother lyrics |||||bobby scott|||||bob russell
who does the voice of nala in the lion king |||||niketa calame|||||sally dworsky|||||moira kelly|||||laura williams
what age do you need to be to buy a bb gun |||||18
who plays auggie in the movie the wonder |||||jacob tremblay
when was the first australian prime minister elected |||||sir edmund barton|||||1901
who plays joker in batman the dark knight |||||ledger
when was the south asian association for regional co-operation (saarc) formed |||||december 1985|||||8 december 1985
when was the first hunger games book published |||||september 14 2008|||||2008
who did bette midler portray in the rose |||||mary rose foster
who wrote cant get you out of my head lyrics |||||cathy dennis and rob davis|||||rob davis|||||cathy dennis
what are the three fifty shades of grey books |||||fifty shades of grey|||||fifty shades darker|||||fifty shades freed
who is the coach for the ottawa senators |||||guy boucher
what is the name of the main artery which takes blood from the heart to the body |||||the aorta|||||aorta
who does eric end up with in that 70s show |||||donna
who wrote papa got a brand new bag |||||james brown
who are nominated for president of india 2017 |||||meira kumar|||||ram nath kovind
where does the path train stop in newark |||||newark penn station
where does route 66 start on the west coast |||||in santa monica
when did they stop cigarette advertising on television |||||january 2 1971|||||1970
points on a sphere or angles in a circle are measured in units called |||||radians
the oligodynamic effect is a phenomenon that describes |||||a biocidal effect of metals
is there a name for the at symbol |||||commercial at|||||at symbol|||||at sign
when did marathon change its name to snickers |||||19 july 1990
who captained the first european ship to sail around the tip of africa |||||bartolomeu dias
who ordered the forbidden city to be built |||||zhu di
who has the power to approve or veto legislation constitution |||||the president
when did amnesia the dark descent come out |||||8 september 2010
what is the oldest street in the philippines |||||cebu city|||||colon street
under article 1 what is the minimum age required to serve in the house of representatives |||||25|||||25 years old
who does demetrius love in a midsummer night dream |||||helena|||||hermia
when did the movie varsity blues come out |||||1999|||||january 15 1999
who was the famous scientist that ran the research lab moseley went to in manchester |||||sir ernest rutherford
what is the meaning of cc and bcc |||||carbon copy|||||carbon copy to secondary recipients|||||blind carbon copy
hazels boyfriend in the fault in our stars |||||augustus waters
when does the day of the dead end |||||november 2
when do you celebrate birthday if born on feb 29 |||||february 29|||||february 28|||||march 1
how many episodes of greys anatomy season 14 |||||24|||||24 episodes|||||21
who is currently serving as president of the senate |||||orrin hatch|||||patrick leahy
who won the champions league final in 2016 |||||real madrid
when was the last episode of vampire diaries aired |||||march 10 2017|||||march 10 2017
where did the jungle book story come from |||||english author rudyard kipling
who played zoe hart on hart of dixie |||||rachel sarah bilson
what was tom hanks character name in castaway |||||chuck noland
when was the minimum wage established in the united states |||||1938|||||1933|||||1912
when did the rational dress society begin to work |||||1881
when was united nations convention on the rights of the child created |||||20 november 1989
who wrote knock knock knocking on heavens door |||||bob dylan
who is the king and queen of the netherlands |||||queen máxima of the netherlands|||||king willemalexander
where is gall bladder situated in human body |||||beneath the liver
who made the song falling in love with you |||||hugo peretti|||||george david weiss|||||elvis presley|||||luigi creatore
who was originally cast to play indiana jones |||||tom selleck
chief ministers of tamil nadu mentioned on wikipedia |||||ramakrishna ranga rao|||||tanguturi prakasam|||||p subbarayan|||||m g ramachandran|||||janaki ramachandran|||||p t rajan|||||j jayalalithaa|||||m karunanidhi|||||a subbarayalu reddiar|||||c n annadurai|||||p s kumaraswamy raja|||||k palaniswami|||||o panneerselvam|||||vr nedunchezhiyan|||||raja of panagal|||||o p ramaswamy reddiyar|||||kurma venkata reddy naidu|||||b munuswamy naidu|||||c rajagopalachari
what is on top of us capitol building |||||the statue of freedom
who is the first president to be impeached |||||andrew johnson|||||johnson
why does kerosene oil rise up in the wick of lantern |||||capillary action
which country has the most coastline in the world |||||canada
who wrote and performed i can only imagine |||||bart millard|||||christian rock band mercyme|||||mercyme
a single period of precession of earth's axis is completed in about |||||approximately 26000 years|||||26000 years
who sang the superbowl national anthem in 2017 |||||luke bryan
what is the name of the hyena in lion king |||||banzai|||||shenzi|||||ed
who played the mad hatter in the batman tv show |||||roddy mcdowall|||||david wayne|||||benedict samuel
when did the united states host the world cup |||||1994
is parallax more pronounced with nearby stars or with distant stars |||||nearby objects|||||nearby

In [ ]:
import json
for i in range(len(selected_skills)):
    file_name = json_file_names[i]
    json_object = json.dumps(json_dicts[i], indent = 4)
    with open(file_name, "w") as outfile:
        outfile.write(json_object)